In [ ]:
"""
This IPython notebook is used for debuging the net forward function. 
"""

%matplotlib inline

import os 
import sys
import numpy as np
import time
import imp
import cv2
import tensorflow as tf

sys.path.append('.')
from datasets.videodb import prepare_trajdb
from datasets.motdb import MotDB
from track_rcnn.net import TrackRCNN
from track_rcnn.layer import roi_pool
from track_rcnn.minibatch import get_minibatch
sys.path.append('./caffe-tensorflow')
from kaffe.tensorflow import Network

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from JSAnimation.IPython_display import display_animation

sess = tf.InteractiveSession()

In [ ]:
# Build
path_config = os.path.join('track_rcnn', 'config.py')
config = imp.load_source('*', path_config)
cfg = config.cfg
m = TrackRCNN(cfg) 

# Initialize
init_scale = cfg.TRAIN.INITIAL_SCALE
initializer = tf.random_uniform_initializer(-init_scale, init_scale)
tf.initialize_all_variables().run()
m.load_cnn_params(sess)

# Experiments

In [ ]:
# Prepare data
train_videos = ['TUD-Stadtmitte', 'ETH-Bahnhof', 'KITTI-13']
# train_videos = ['KITTI-13']

motdb = MotDB(train_videos)
motdb.load_trajdb()
motdb.create_rois_for_trajdb(cfg)
trajdb = prepare_trajdb(cfg, motdb.trajdb)

print("#train_trajectories: %d" % len(trajdb))
trajdb = list(trajdb)

In [ ]:
# Get batch
batch_size = m.n_trajs
perm = np.arange(len(trajdb))
np.random.shuffle(perm)
i = 0
index = perm[i*batch_size:(i+1)*batch_size]
_trajdb = [trajdb[idx] for idx in index]

# Run forward
feed_dict = get_minibatch(cfg, _trajdb, m, random=True)
ims = feed_dict[m.im_input]
rois = feed_dict[m.rois_input]
scores = feed_dict[m.scores_target]
print rois.shape
# [conv5, roi_pool] = sess.run(m.conv5, feed_dict=feed_dict)
[conv5, roi_pool, fc7, output] = sess.run([m.conv5, m.roi_pool, m.fc7, m.output], feed_dict=feed_dict)

In [ ]:
# Visualization

n_channels = 30
channel = 256
t = 2
for roi_idx in range(cfg.TRAIN.BATCH_SIZE):
    fig = plt.figure(figsize=(20, 3))
    ax1 = fig.add_subplot(1, 4, 1)
    ax2 = fig.add_subplot(1, 4, 2)
    ax3 = fig.add_subplot(1, 4, 3)
    ax4 = fig.add_subplot(1, 4, 4)
    
    # Plot image
    im_idx = int(rois[roi_idx, t, 0])
    im = ims[im_idx, t, :, :, :] + cfg.PIXEL_MEANS
    ax1.imshow(cv2.cvtColor(im.astype('uint8'), cv2.COLOR_BGR2RGB))
    x1, y1, x2, y2 = rois[roi_idx, t, 1:]
    ax1.add_patch(patches.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, linewidth=1, edgecolor='r'))
    ax1.set_title("%.1f, %.1f, %.1f, %.1f" % ((x1+x2)/2, (y1+y2)/2, x2-x1, y2-y1))
            
    # Plot conv5 feature (mean of multiple channels)
    ax2.matshow(np.mean(conv5[im_idx, t, :, :, :n_channels], axis=2), cmap='Greys_r')
    # ax2.matshow(conv5[im_idx, t, :, :, channel], cmap='Greys_r')
    x1, y1, x2, y2 = rois[roi_idx, t, 1:] * m.spatial_scale
    ax2.add_patch(patches.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, linewidth=1, edgecolor='r'))
    ax2.set_title("%.1f, %.1f, %.1f, %.1f" % ((x1+x2)/2, (y1+y2)/2, x2-x1, y2-y1))
    
    # Plot the pool5
    ax3.matshow(np.mean(roi_pool[roi_idx, t, :, :, :n_channels], axis=2), cmap='Greys_r')
    # ax3.matshow(roi_pool[roi_idx, t, :, :, channel], cmap='Greys_r')
    
    # Plot the fc7
    ax4.matshow(np.reshape(fc7[roi_idx, t, :], (64, 64)), cmap='Greys_r')
    ax4.set_title("score %.1f" % (scores[roi_idx, t]))